1 Face Recognition using CNN

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import TensorBoard
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score
from keras.utils import np_utils
import itertools

### 2 Step2:
• Load Dataset :

After loading the Dataset we have to normalize every image.
Note: an image is a Uint8 matrix of pixels and for calculation, you need to convert the format of
the image to float or double

In [ ]:
#load dataset
data = np.load('ORL_faces.npz')
# load the "Train Images"
x_train = data['trainX']
#normalize every image
x_train = np.array(x_train,dtype='float32')/255
x_test = data['testX']
x_test = np.array(x_test,dtype='float32')/255
# load the Label of Images
y_train= data['trainY']
y_test= data['testY']
# show the train and test Data format
print('x_train : {}'.format(x_train[:]))
print('Y-train shape: {}'.format(y_train))
print('x_test shape: {}'.format(x_test.shape))

### 3 Step 3
Split DataSet : Validation data and Train

Validation DataSet: this data set is used to minimize overfitting.If the accuracy over the training
data set increases, but the accuracy over then validation data set stays the same or decreases, then
we’re overfitting your neural network and you should stop training.

• Note: we usually use 30 percent of every dataset as the validation data but Here we only used
5 percent because the number of images in this dataset is very low.

In [ ]:
x_train, x_valid, y_train, y_valid= train_test_split(
x_train, y_train, test_size=.05, random_state=1234,)

### 4 Step 4
for using the CNN, we need to change The size of images ( The size of images must be the same)

In [ ]:
im_rows=112
im_cols=92
batch_size=512
im_shape=(im_rows, im_cols, 1)
#change the size of images
x_train = x_train.reshape(x_train.shape[0], *im_shape)
x_test = x_test.reshape(x_test.shape[0], *im_shape)
x_valid = x_valid.reshape(x_valid.shape[0], *im_shape)
print('x_train shape: {}'.format(y_train.shape[0]))
print('x_test shape: {}'.format(y_test.shape))

### 5 Step 5
Build CNN model: CNN have 3 main layer: * 1-Convolotional layer * 2- pooling layer
* 3- fully connected layer
we could build a new architecture of CNN by changing the number and position of layers.

In [ ]:
#filters= the depth of output image or kernels
cnn_model= Sequential([
Conv2D(filters=36, kernel_size=7, activation='relu', input_shape= im_shape),
MaxPooling2D(pool_size=2),
Conv2D(filters=54, kernel_size=5, activation='relu', input_shape= im_shape),
MaxPooling2D(pool_size=2),
Flatten(),
Dense(2024, activation='relu'),
Dropout(0.5),
Dense(1024, activation='relu'),
Dropout(0.5),
Dense(512, activation='relu'),
Dropout(0.5),
#20 is the number of outputs
3
Dense(20, activation='softmax')
])
cnn_model.compile(
loss='sparse_categorical_crossentropy',#'categorical_crossentropy',
optimizer=Adam(lr=0.0001),
metrics=['accuracy'])

### 6 Step 6
Train the Model
• Note: We can change the number of epochs

In [ ]:
: history=cnn_model.fit(
np.array(x_train), np.array(y_train), batch_size=512,
epochs=250, verbose=2,
validation_data=(np.array(x_valid),np.array(y_valid)))

In [ ]:
scor = cnn_model.evaluate( np.array(x_test), np.array(y_test), verbose=0)
print('test los {:.4f}'.format(scor[0]))
print('test acc {:.4f}'.format(scor[1]))

### 7 Step 7
plot the result

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### 8 step 8
Plot Confusion Matrix

In [ ]:
predicted =np.array( cnn_model.predict(x_test))
#print(predicted)
#print(y_test)
ynew = cnn_model.predict_classes(x_test)
Acc=accuracy_score(y_test, ynew)
print("accuracy : ")
print(Acc)
#/tn, fp, fn, tp = confusion_matrix(np.array(y_test), ynew).ravel()
cnf_matrix=confusion_matrix(np.array(y_test), ynew)
y_test1 = np_utils.to_categorical(y_test, 20)
def plot_confusion_matrix(cm, classes,
normalize=False,
17
title='Confusion matrix',
cmap=plt.cm.Blues):
"""
This function prints and plots the confusion matrix.
Normalization can be applied by setting `normalize=True`.
"""
if normalize:
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#print("Normalized confusion matrix")
else:
print('Confusion matrix, without normalization')
#print(cm)
plt.imshow(cm, interpolation='nearest', cmap=cmap)
plt.title(title)
plt.colorbar()
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)
fmt = '.2f' if normalize else 'd'
thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
plt.text(j, i, format(cm[i, j], fmt),
horizontalalignment="center",
color="white" if cm[i, j] > thresh else "black")
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()
print('Confusion matrix, without normalization')
print(cnf_matrix)
plt.figure()
plot_confusion_matrix(cnf_matrix[1:10,1:10], classes=[0,1,2,3,4,5,6,7,8,9],
title='Confusion matrix, without normalization')
plt.figure()
plot_confusion_matrix(cnf_matrix[11:20,11:20],␣
,→classes=[10,11,12,13,14,15,16,17,18,19],
title='Confusion matrix, without normalization')
print("Confusion matrix:\n%s" % confusion_matrix(np.array(y_test), ynew))
print(classification_report(np.array(y_test), ynew))